# Run Backtest Pipeline

This notebook demonstrates running the backtest pipeline step by step.

In [ ]:
import sys
sys.path.append('..')

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, time, timedelta

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
RESULTS_DIR = os.path.join(PROJECT_ROOT, 'results')
DATA_CACHE_DIR = os.path.join(PROJECT_ROOT, 'data_cache')
LOGS_DIR = os.path.join(PROJECT_ROOT, 'logs')
CONFIG_DIR = os.path.join(PROJECT_ROOT, 'config')

os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(DATA_CACHE_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

from src.data.data_loader import DataLoader
from src.data.data_processor import DataProcessor
from src.indicators.bollinger_bands import add_bollinger_bands
from src.indicators.rsi import add_rsi
from src.indicators.atr import add_atr
from src.strategy.signal_generator import SignalGenerator
from src.strategy.risk_manager import RiskManager
from src.backtest.backtest_engine import BacktestEngine
from src.backtest.performance import PerformanceMetrics
from src.visualization.backtest import plot_backtest_results, plot_performance_metrics, plot_equity_curve, plot_trade_analysis

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 8)

## Step 1: Load Market Data

In [ ]:
from src.optimization.config_loader import ConfigLoader

config_loader = ConfigLoader()
config = config_loader.get_config()

params = config_loader.get_default_parameters()
bb_window = params.get("bb_window", 20)
bb_std = params.get("bb_std", 1.8)
rsi_period = params.get("rsi_period", 13)
rsi_lower = params.get("rsi_lower", 30)
rsi_upper = params.get("rsi_upper", 70)
atr_period = params.get("atr_period", 14)
take_profit_mult = params.get("take_profit_mult", 4.0)
stop_loss_mult = params.get("stop_loss_mult", 1.0)
timeframe = params.get("default_timeframe", "15min")

print(f"\nStrategy Parameters:")
print(f"Timeframe: {timeframe}")
print(f"Bollinger Bands: Window={bb_window}, StdDev={bb_std}")
print(f"RSI: Period={rsi_period}, Oversold={rsi_lower}, Overbought={rsi_upper}")
print(f"ATR: Period={atr_period}")
print(f"Take Profit: {take_profit_mult}x ATR")
print(f"Stop Loss: {stop_loss_mult}x ATR")

## Step 2: Process Data and Add Indicators

In [ ]:
start_date = '2024-01-01'
end_date = '2024-06-01'

loader = DataLoader(cache_dir=DATA_CACHE_DIR)
processor = DataProcessor(cache_dir=os.path.join(DATA_CACHE_DIR, "ohlcv"))

print(f"Loading market data from {start_date} to {end_date}...")
data = loader.get_active_contract_data(start_date, end_date)
print(f"Loaded {len(data)} tick data points")

print(f"\nResampling to {timeframe} candles...")
ohlcv_data = processor.resample_to_ohlcv(data, timeframe=timeframe)
print(f"Created {len(ohlcv_data)} OHLCV candles")

print("\nSample OHLCV data:")
display(ohlcv_data.head())

## Step 3: Generate Trading Signals

In [ ]:
from src.pipeline import TradingPipeline
pipeline = TradingPipeline(config)

print("Adding technical indicators...")
prepared_data = pipeline.prepare_data(data, timeframe=timeframe)

print("\nData with indicators:")
display(prepared_data.tail())

In [ ]:
from src.visualization.backtest import plot_candles

window_size = 200
if len(prepared_data) > window_size:
    plot_window = prepared_data.iloc[-window_size:]
else:
    plot_window = prepared_data

print(f"Plotting candlestick chart for the last {len(plot_window)} candles...")
fig = plot_candles(
    plot_window, 
    ticker="VN30F1M",
    volume=True,
    save_path=os.path.join(RESULTS_DIR, 'candlestick_chart.png')
)
plt.show()


## Step 4: Run Backtest

In [ ]:
print("Generating trading signals...")
signal_gen = SignalGenerator(rsi_lower=rsi_lower, rsi_upper=rsi_upper)
signals_df = signal_gen.generate_signals(prepared_data)

buy_signals = signals_df['buy_signal'].sum()
sell_signals = signals_df['sell_signal'].sum()
print(f"Generated {buy_signals} buy signals and {sell_signals} sell signals")

buy_examples = signals_df[signals_df['buy_signal'] == 1].head(3)
sell_examples = signals_df[signals_df['sell_signal'] == 1].head(3)

print("\nSample buy signals:")
display(buy_examples[['datetime', 'close', 'rsi', 'lower_band', 'upper_band']])

print("\nSample sell signals:")
display(sell_examples[['datetime', 'close', 'rsi', 'lower_band', 'upper_band']])

## Step 5: Analyze Performance

In [ ]:
risk_manager = RiskManager()
backtest_engine = BacktestEngine(initial_balance=100000)

print("Running backtest...")
backtest_results = backtest_engine.run_backtest(signals_df, risk_manager)

if 'trades' in backtest_results and not backtest_results['trades'].empty:
    n_trades = len(backtest_results['trades'])
    final_balance = backtest_results['final_balance']
    print(f"Backtest completed with {n_trades} trades")
    print(f"Initial balance: $100,000")
    print(f"Final balance: ${final_balance:.2f}")
    print(f"Net P&L: ${final_balance - 100000:.2f}")
else:
    print("No trades were executed in the backtest period")

## Step 6: Visualize Results

In [ ]:
print("Plotting backtest results with signals...")
fig = plot_backtest_results(
    signals_df,
    trades_df=backtest_results['trades'] if 'trades' in backtest_results and not backtest_results['trades'].empty else None,
    indicators=True,
    figsize=(18, 14),
    save_path=os.path.join(RESULTS_DIR, 'backtest_chart.png')
)
plt.show()

In [ ]:

if 'trades' in backtest_results and not backtest_results['trades'].empty:
    performance = PerformanceMetrics(
        backtest_results['trades'], 
        backtest_results['portfolio_history']
    )
    
    metrics = performance.generate_report()
    
    print("Performance Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value:.4f}" if isinstance(value, (float, int)) else f"{key}: {value}")
    
    fig = plot_performance_metrics(metrics)
    plt.show()
else:
    print("No performance metrics available - no trades were executed")

## Step 7: Save Results

In [ ]:


if 'portfolio_history' in backtest_results and len(backtest_results['portfolio_history']) > 1:
    fig = plot_equity_curve(backtest_results['portfolio_history'], save_path=os.path.join(RESULTS_DIR, 'equity_curve.png'))
    plt.show()
else:
    print("No equity curve available - insufficient data")

In [ ]:


if 'trades' in backtest_results and not backtest_results['trades'].empty:
    trades_df = backtest_results['trades']
    
    fig_trades, fig_exit, profit_by_exit = plot_trade_analysis(trades_df)
    
    plt.figure(fig_trades.number)
    plt.show()
    
    if fig_exit:
        plt.figure(fig_exit.number)
        plt.show()
    
    if profit_by_exit is not None:
        print("\nProfit by Exit Reason:")
        display(profit_by_exit)
else:
    print("No trade distribution available - no trades were executed")

In [ ]:
if 'trades' in backtest_results and not backtest_results['trades'].empty:
    trades_df = backtest_results['trades']
    trades_df.to_csv(os.path.join(RESULTS_DIR, 'trades.csv'), index=False)
    print(f"Saved {len(trades_df)} trades to {os.path.join(RESULTS_DIR, 'trades.csv')}")
    
    if 'metrics' in locals():
        metrics_dict = {k: float(v) if isinstance(v, (int, float, np.number)) else v for k, v in metrics.items()}
        
        result_data = {
            "parameters": params,
            "test_period": {
                "start_date": start_date,
                "end_date": end_date
            },
            "metrics": metrics_dict,
            "summary": {
                "initial_balance": 100000,
                "final_balance": float(backtest_results['final_balance']),
                "total_trades": len(trades_df),
                "buy_signals": int(buy_signals),
                "sell_signals": int(sell_signals),
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
        }
        
        with open(os.path.join(RESULTS_DIR, 'backtest_results.json'), 'w') as f:
            json.dump(result_data, f, indent=4)

        print(f"Saved performance metrics to {os.path.join(RESULTS_DIR, 'backtest_results.json')}")
else:
    print("No results saved - no trades were executed")